In [16]:
import numpy as np
from keras.models import load_model, Model
from keras.preprocessing.image import ImageDataGenerator
# from keras.applications import inception_resnet_v2
# from keras.layers import Dense, Dropout
import pandas as pd
from src.utils import printModelMetric, analyze
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
model1 = load_model("inception_v4_finetune_84.h5")
#model2 = load_model("inception_v3_finetune_828.h5")
#model3 = load_model("inception_v3_85_hinge.h5")
#model3 = load_model("inception_v4_difftop.h5")
model3 = load_model("inception_v4_hinge.h5")

c:\users\tomforge\anaconda3\envs\aicv\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
TEST_DIR = 'TestImages'
IMG_WIDTH, IMG_HEIGHT = 256, 256
NUM_PREDICT = 16111

# Need to apply the same data preprocessing
test_datagen = ImageDataGenerator(
    rescale = 1./255)

# Set no shuffle so the predictions follow file order
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    class_mode = "categorical",
    shuffle = False)

        

Found 16111 images belonging to 1 classes.


In [3]:
TEST_DIR = 'data/test'
IMG_WIDTH, IMG_HEIGHT = 256, 256
NUM_PREDICT = 3813

# Need to apply the same data preprocessing
test_datagen = ImageDataGenerator(
    rescale = 1./255)

# Set no shuffle so the predictions follow file order
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    class_mode = "categorical",
    shuffle = False)

        

Found 3813 images belonging to 18 classes.


In [18]:
results1 = model1.predict_generator(test_generator, verbose=True)

120/120 [==============================] - 60s 500ms/step


In [21]:
model2 = load_model("inception_v3_finetune_828.h5")

In [22]:
results2 = model2.predict_generator(test_generator, verbose=True)

120/120 [==============================] - 38s 316ms/step


In [19]:
results3 = model3.predict_generator(test_generator, verbose=True)

120/120 [==============================] - 59s 494ms/step


In [24]:
model4 = load_model("inception_v4_difftop_832.h5")

In [29]:
results4 = model4.predict_generator(test_generator, verbose=True)

120/120 [==============================] - 64s 534ms/step


In [13]:
ensemble = np.argmax((results1 + results3)/2, axis=1)
ensemble_w828 = np.argmax((results1 + results2 + results3)/3, axis=1)

# womenPreds = np.argmax(womenRes, axis=1)
# correctedWomenPreds = np.copy(womenPreds)
# ensemble12 = np.argmax((results1 + results2)/2, axis=1)
# ensemblewWomen = np.copy(ensemble12)
# womenCat = [5, 7, 9, 11, 13, 15, 16]
# womenMask = np.full_like(womenPreds, False, dtype=bool)
# for i,v in enumerate(womenCat):
#     # Get all the rows predicted as women tops
#     if v != 16:
#         womenMask = womenMask | (ensemblewWomen == v)
#     # Update the women classifier labels to the correct ones
#     correctedWomenPreds[womenPreds == i] = v
# # Use women classfier values for women tops
# ensemblewWomen[womenMask] = correctedWomenPreds[womenMask]
# print(np.bincount(ensemblewWomen))
# print(np.bincount(ensemble12))

In [30]:
# printModelMetric(analyze(pred_labels1), "inception_v4_84")
# printModelMetric(analyze(pred_labels2), "inception_v3_828") 
# printModelMetric(analyze(ensemble12), "inception_v4_84 + inception_v3_828 ensemble")
# printModelMetric(analyze(ensemblewWomen), "Ensemble + women classifier")
#printModelMetric(analyze(np.argmax(results3, axis=1)), "inception_v4_hinge")
ensemble123 = np.argmax((results1 + results2 + results3)/3, axis=1)
printModelMetric(analyze(ensemble123), "ensemble w hinge no difftop")
ensemble124 = np.argmax((results1 + results2 + results4)/3, axis=1)
printModelMetric(analyze(ensemble123), "ensemble no hinge w difftop")
ensemble1234 = np.argmax((results1 + results2 + results3 + results4)/4, axis=1)
printModelMetric(analyze(ensemble1234), "ensemble w hinge w difftop")

-----ensemble w hinge no difftop-----
Label 0: Num images: 338; True pos: 336; False pos: 2
>>> Rate truepos: 0.9940828402366864; Rate falsepos: 0.005917159763313609
>>> Most frequently predicted as label 1 with 1 images.
Label 1: Num images: 265; True pos: 265; False pos: 2
>>> Rate truepos: 1.0; Rate falsepos: 0.00749063670411985
>>> Most frequently predicted as label 0 with 0 images.
Label 2: Num images: 192; True pos: 188; False pos: 3
>>> Rate truepos: 0.9791666666666666; Rate falsepos: 0.015706806282722512
>>> Most frequently predicted as label 3 with 1 images.
Label 3: Num images: 191; True pos: 166; False pos: 13
>>> Rate truepos: 0.8691099476439791; Rate falsepos: 0.07262569832402235
>>> Most frequently predicted as label 4 with 10 images.
Label 4: Num images: 197; True pos: 194; False pos: 15
>>> Rate truepos: 0.9847715736040609; Rate falsepos: 0.07177033492822966
>>> Most frequently predicted as label 0 with 1 images.
Label 5: Num images: 211; True pos: 145; False pos: 36
>>

In [10]:
# Extract image ids from the filenames (which are in order)
ids = [int(f[10:-4]) for f in test_generator.filenames]
# Create the csv for submission
sub = pd.DataFrame({'id':ids, 'category':ensemble})
sub = sub.sort_values(by=['id'])
# Place id column first
sub = sub[['id', 'category']]
sub.to_csv('2semble_w_hinge.csv', index=False)

In [11]:
# Create the csv for submission
sub = pd.DataFrame({'id':ids, 'category':ensemble_w828})
sub = sub.sort_values(by=['id'])
# Place id column first
sub = sub[['id', 'category']]
sub.to_csv('3semble_w_hinge.csv', index=False)